In [ ]:
import gymnasium as gym
from stable_baselines3 import SAC
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv, VecMonitor, VecNormalize
from stable_baselines3.common.vec_env.base_vec_env import VecEnv, VecEnvStepReturn, VecEnvWrapper
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from IPython.display import clear_output

In [ ]:
class VecPendulumRewardWrapper(VecEnvWrapper):
    def __init__(self, venv: VecEnv):
        super().__init__(venv=venv)

    def reset(self) -> np.ndarray:
        obs = self.venv.reset()
        return obs
    
    def step_async(self, actions: np.ndarray) -> None:
        self.venv.step_async(actions)

    def step_wait(self) -> VecEnvStepReturn:
        obs, reward, done, info = self.venv.step_wait()
        reward = 1/obs[:, 0]**2 + 1/obs[:, 1]**2
        return obs, reward, done, info

def make_env(env_id, rank, seed=0):
    def _init():
        env = gym.make(env_id)
        return env
    return _init

env_id = 'InvertedPendulum-v4'  # Replace with your MuJoCo environment
num_envs = 16  # Number of parallel environments

# Create the vectorized environment
env = SubprocVecEnv([make_env(env_id, i) for i in range(num_envs)])
print(env.seed())
# Add the reward wrapper
env = VecPendulumRewardWrapper(env)
env = VecMonitor(env)  # Optional: for monitoring and logging

model = SAC('MlpPolicy', env, verbose=0)
model.learn(total_timesteps=1e5, log_interval=4, progress_bar=True)
model.save("sac_pendulum")

# del model # remove to demonstrate saving and loading



In [ ]:
model = SAC.load("sac_pendulum")

class PendulumRewardWrapper(gym.Wrapper):
    def __init__(self, env):
        super().__init__(env=env)

    def reset(self):
        obs = self.env.reset()
        return obs

    def step(self, action):
        obs, reward, terminated, truncated, info = self.env.step(action)
        reward = 1/obs[0]**2 + 1/obs[1]**2
        return obs, reward, terminated, truncated, info

env = gym.make("InvertedPendulum-v4", render_mode='rgb_array')
env = PendulumRewardWrapper(env)
obs, info = env.reset()

data = []
total_reward = 0
terminated = False
truncated = False
# while not (terminated or truncated):
while not truncated:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    total_reward += reward
    data.append(np.concatenate((action, obs, np.array([total_reward]))))
    # clear_output(wait=True)
    # plt.imshow( env.render() )
    # plt.axis('off')
    # plt.show()

names = ["action", "cart_pos", "pole_angle", "cart_velocity", "pole_ang_vel", "reward"]

P = pd.DataFrame(data, columns = names)

name = "testrun"
saveFile = "recordings/" + name
P.to_csv(saveFile + ".csv")

In [ ]:
obs, info = env.reset()
while True:
    action, _states = model.predict(obs, deterministic=True)
    obs, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
        obs, info = env.reset()
    clear_output(wait=True)
    plt.imshow( env.render() )
    plt.axis('off')
    plt.show()